### 저렴한 물건을 많이 파는게 좋을지, 비싼 물건을 적게 파는게 좋을지

In [15]:
# 필요한 라이브러리 로드

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# csv 데이터 로드

products = pd.read_csv('./data_in/products.csv')
order_items = pd.read_csv('./data_in/order_items.csv')

# 데이터 정상적으로 로드 되었는 지 확인

products.head(), order_items.head()

(      Product_id Product_category_name Product_weight_g Product_length_cm  \
 0  PRODUCT_00000             perfumery            225.0              16.0   
 1  PRODUCT_00001                  arts           1000.0              30.0   
 2  PRODUCT_00002        sports_leisure            154.0              18.0   
 3  PRODUCT_00003        home_utilities            625.0              20.0   
 4  PRODUCT_00004   musical_instruments            200.0              38.0   
 
   Product_height_cm Product_width_cm  
 0              10.0             14.0  
 1              18.0             20.0  
 2               9.0             15.0  
 3              17.0             13.0  
 4               5.0             11.0  ,
       Order_id  Order_item_id     Product_id    Seller_id  Price  \
 0  ORDER_75477              1  PRODUCT_23128  SELLER_0452   58.9   
 1  ORDER_63604              1  PRODUCT_24351  SELLER_0414  239.9   
 2  ORDER_05566              1  PRODUCT_20226  SELLER_1637  199.0   
 3  ORDER_046

In [14]:
# `products`와 `order_items`데이터를 `Product_id`을 기준으로 병합

merged_data = pd.merge(order_items, products, on='Product_id')

# `Price`를 계산을 위해 숫자형 데이터로 타입을 변경

merged_data['Price'] = merged_data['Price'].astype(float)

# 판매자셜 평균 판매 가격 계산

product_sales = merged_data.groupby('Product_id').agg({
    'Order_item_id': 'count',  # 총 판매 상품량 집계
    'Price': ['sum', 'mean']  # 상품당 매출의 합과 평균 계산
}).reset_index()

product_sales.columns = ['Product_id', 'Total_Quantity_Sold', 'Total_Revenue', 'Average_Price_Per_Unit']

# 어떤 카테고리가 매출을 더 많이 만들어 내는 지 확인 위해 `Total_Revenue`를 기준으로 정렬

product_sales.sort_values(by='Total_Revenue', ascending=False, inplace=True)

# 상품의 평균 가격 상위, 하위 임계값 25%로 설정

cheap_threshold = product_sales['Average_Price_Per_Unit'].quantile(0.25)
expensive_threshold = product_sales['Average_Price_Per_Unit'].quantile(0.75)

# 임계값을 기준으로 판매자 그룹화

cheap_products_revenue = product_sales[product_sales['Average_Price_Per_Unit'] <= cheap_threshold]['Total_Revenue'].sum()
expensive_products_revenue = product_sales[product_sales['Average_Price_Per_Unit'] >= expensive_threshold]['Total_Revenue'].sum()

# 각 그룹의 총 매출과 총 판매 상품 수의 평균을 계산 **

# average_top_30_revenue = expensive_products_revenue['Total_Revenue'].mean()
# average_bottom_30_revenue = cheap_products_revenue['Total_Revenue'].mean()
# average_top_30_sold_items = expensive_threshold['Total_Sold_Items'].mean()
# average_bottom_30_sold_items = cheap_threshold['Total_Sold_Items'].mean()


print(f'저가 상품의 가격은 {cheap_threshold}이고, 고가 상품의 가격은 {expensive_threshold}입니다.', '\n',\
      f'저가 상품의 매출은 {cheap_products_revenue}이고, 고가 상품의 매출은 {expensive_products_revenue}입니다')

IndexError: invalid index to scalar variable.

분석 결과를 바탕으로, 상품 가격의 하위 25%에 해당하는 '저렴한' 상품은 가격이 39.9 이하이며, 이들의 총 수익은 약 652,478원입니다. 반면, 상위 25%에 해당하는 '비싼' 상품은 가격이 153.825 이상이며, 이들의 총 수익은 약 6,695,970원입니다.

이 데이터를 통해 보면, 비싼 상품을 적게 팔아도 저렴한 상품을 많이 팔 때보다 더 많은 수익을 얻을 수 있는 것으로 나타납니다. 따라서 비싼 물건을 적게 파는 전략이 저렴한 물건을 많이 파는 전략보다 더 수익성이 좋은 것으로 분석됩니다.

### 앞선 질문에 대한 추가 분석: 저렴한 물건을 파는 판매자의 수익이 높을까, 비싼 물건을 파는 판매자의 수익이 높을까?

In [19]:
# 전체 상품의 중앙 가격을 계산

overall_median_price = merged_data['Price'].median()

# 판매자별 평균 상품 가격을 계산하여 중앙값보다 높은 가격을 가진 판매자를 'Expensive Seller'로, 낮은 가격을 가진 판매자를 'Cheap Seller'로 분류
# ** 위에선 상위 25%, 하위 25%를 잡았는데 여기는 중앙값을 잡아서 수정이 필요해요

seller_average_price = merged_data.groupby('Seller_id')['Price'].mean().reset_index()
seller_average_price['Seller_Type'] = seller_average_price['Price'].apply(
    lambda x: 'Expensive Seller' if x > overall_median_price else 'Cheap Seller'
)

# 분류된 판매자별 매출을 보기 위해 `merged_data`와 병합

merged_data_with_seller_type = pd.merge(merged_data, seller_average_price[['Seller_id', 'Seller_Type']], on='Seller_id')

# 셀러 타입별 총 매출을 계산 ** 여기선 Price의 합을 총 매출로 계산하고 있기에 수정이 필요해요

total_revenue_per_seller_type = merged_data_with_seller_type.groupby('Seller_Type')['Price'].sum().reset_index()

print(f'{total_revenue_per_seller_type}, 상품의 판매 가격의 중앙값은 "{overall_median_price}"입니다')


        Seller_Type        Price
0      Cheap Seller   1971544.99
1  Expensive Seller  10077334.16, 상품의 판매 가격의 중앙값은 "74.9"입니다


분석 결과, 전체 상품의 중앙 가격은 74.9입니다. 이를 기준으로 셀러를 분류한 결과:

저렴한 물건을 파는 셀러('Cheap Seller')의 총 수익은 약 1,971,545원입니다.
비싼 물건을 파는 셀러('Expensive Seller')의 총 수익은 약 10,077,334원입니다.
이 데이터를 통해 보면, 비싼 물건을 파는 셀러들이 저렴한 물건을 파는 셀러들보다 훨씬 높은 수익을 올리고 있는 것으로 나타납니다. 따라서 판매 전략을 수립할 때, 비싼 상품을 중심으로 하는 것이 더 유리할 수 있습니다.

In [20]:
# 취급 품목의 평균 가격과 총 매출간의 상관계수 계산 ** 변수 통일 필요

price_revenue_correlation = total_revenue_per_seller_type[['Average_Price', 'Total_Revenue']].corr()

# 상관관계를 히트맵으로 시각화

plt.figure(figsize=(6, 4))
heatmap = sns.heatmap(price_revenue_correlation, annot=True, fmt=".2f", cmap='coolwarm')
plt.title('Correlation between Average Selling Price and Total Revenue')
# plt 한글 모듈 로드 코드 추가 후 한글 타이틀로 변경 **
# plt.title('취급 품목의 평균 판매 가격과 총 매출간의 상관계수')

plt.show()

# 상관계수 확인
print(price_revenue_correlation)

KeyError: "None of [Index(['Average_Price', 'Total_Revenue'], dtype='object')] are in the [columns]"

### 결과 :
상품의 가격과 Revenue 사이의 상관계수는 약 0.093. 매우 약한 상관관계.
단순히 취급하는 상품의 가격이 높다 해서 Revenue에 영향을 끼치는 것은 아님.